In [ ]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import datetime as dt
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
#read from fundamentals.csv
#fundData = pd.read_csv("../input/fundamentals.csv")
priceData = pd.read_csv("../input/prices.csv")

#rename column name to prepare for merging
#fundData = fundData.rename(columns={'Period Ending': 'date'})
#fundData = fundData.rename(columns={'Ticker Symbol': 'symbol'})

#print(fundData.head())
#print("________")
#print(priceData.head())

#innerjoin the fund table to the price table, since we only data that is complete 
#mergedTable = pd.merge(fundData, priceData, how='inner', on=['date', 'symbol'])
#fill up all the NA fields
#mergedTable = mergedTable.fillna(0)
#mergedTable.to_csv('merged.csv')
le = preprocessing.LabelEncoder()
#mergedTable = priceData["symbol"=="WLTW"]
mergedTable = priceData.loc[priceData['symbol'] == "WLTW"]

#print(mergedTable)
print("________")

#then split into predictors and target sets
targetSet = mergedTable.iloc[:,-4:-3]
predictorSet = mergedTable.iloc[:,0:1]

listP = ["epoch"]
#encode the ticker string to integer
#predictorSet["symbol"] = le.fit_transform(predictorSet["symbol"])
#encode date
predictorSet["date"] = pd.to_datetime(predictorSet['date'])
predictorSet['epoch'] = (predictorSet['date'] - dt.datetime(1970,1,1)).dt.total_seconds()
#then drop date column
#predictorSet = predictorSet.drop(['date'], axis=1)

#print(targetSet)
#print(predictorSet)
print("________")

trainTargetData = targetSet[:200]
trainPredictorData = predictorSet[:200]

testTargetData = targetSet[200:]
testPredictorData = predictorSet[200:]
print(trainTargetData.head())
print(trainPredictorData.head())
print(testTargetData.head())
print(testPredictorData.head())

#print(testPredictorData[listP])
#print("Contains NA values:"+str(trainData.isnull().any().any()))

#finally..
lm = LinearRegression()
lm.fit(trainPredictorData,trainTargetData) 
predictions = lm.predict(testPredictorData[listP])
#print(predictions)
predictions2 = predictions.copy()

test = testTargetData.copy()
#match actual and predicted ones
test['predicted'] = predictions2
print(test)


fig, ax = plt.subplots()
ax.scatter(testPredictorData["epoch"], predictions)
#ax.plot([testPredictorData["epoch"].min(),testPredictorData["epoch"].max()],[testTargetData["close"].min(),testTargetData["close"].max()], 'r:')
ax.plot(testPredictorData["epoch"], testTargetData["close"], 'r:')
ax.set_xlabel('date')
ax.set_ylabel('price')
plt.show()

print(lm.score(testPredictorData, testTargetData))
    
#scores = cross_val_score(lm, iris.data, iris.target, cv=5)

#print("training set size:"+trainData.shape)
#print("training set size:"+testData.shape)
#print(trainData)